### Introduction

In this file, we first extract and combine the datasets for our use in prediction.

At the end, we export our variables to multiple csv files for later import, for the actual modeling.

In [1]:
# load necessary libraries
import pandas as pd
import numpy as np
import datetime as dt
from sklearn.preprocessing import LabelEncoder

if any of the libraries above is not present in your system, please install them by using this format, in your Jupyter Notebook:

`!pip install your_library [--user]`

In [3]:
# Load all MLB data and combine/merge them into one table

Note: the following code will work only if "archive" folder is present in the same directory with this Jupyter notebook.

Archive folder can be downloaded from [here](https://www.kaggle.com/pschale/mlb-pitch-data-20152018).

In [2]:
pitches = pd.read_csv("archive/pitches.csv")

In [3]:
atbats = pd.read_csv('archive/atbats.csv')

Note: the following games.csv contains some additional columns/features that were not present in the original games.csv file; it is uploaded on [GitHub](https://github.com/yshong211/mlb_ant) separately.

In [4]:
games = pd.read_csv('games.csv') 

In [5]:
names = pd.read_csv('archive/player_names.csv')

In [6]:
# Merging pitches and atbats by ab_id
data = pd.merge(pitches, atbats, on = 'ab_id', how = 'left')

In [7]:
# Merging games data with pitches and atbats by g_id
data = pd.merge(data, games, on = 'g_id', how = 'left')

In [8]:
year = data.g_id.astype('str')

In [9]:
# Since the year is the first four characters, we will isolate those characters
year = year.str[:4]

In [10]:
data['year'] = year.values

For our purposes, we first will be using the data only the years of 2017 and 2018.

In [11]:
data1718 = data[(data['year'] == '2017') | (data['year'] == '2018')]

In [12]:
names.head(5)

,id,first_name,last_name
0,452657,Jon,Lester
1,425794,Adam,Wainwright
2,457435,Phil,Coke
3,435400,Jason,Motte
4,519166,Neil,Ramirez


In [13]:
data1718 = pd.merge(data1718, names, left_on = 'pitcher_id', right_on = 'id')

In [14]:
data1718.to_csv("data1718.csv")

start from below, if data1718.csv is ready

In [15]:
data1718 = pd.read_csv("data1718.csv")

In [16]:
data1718_merge = pd.DataFrame(data1718[['pitch_type', 'pitcher_id', 'year']])

In [17]:
# drop the outliers
data1718_merge = data1718_merge[data1718_merge.pitch_type != 'AB']
data1718_merge = data1718_merge[data1718_merge.pitch_type != 'UN']
data1718_merge = data1718_merge[data1718_merge.pitch_type != 'EP']
data1718_merge = data1718_merge[data1718_merge.pitch_type != 'SC']
data1718_merge = data1718_merge[data1718_merge.pitch_type != 'PO']
data1718_merge = data1718_merge[data1718_merge.pitch_type != 'FO']

In [18]:
data1718_merge.dropna(inplace=True)

In [19]:
data1718_merge.pitch_type.value_counts()

FF    507473
SL    240439
FT    177897
CH    146135
CU    118919
SI    108393
FC     75883
KC     35470
FS     22126
KN      3369
Name: pitch_type, dtype: int64

In [20]:
pitcherid=set()
for i in data1718_merge.pitcher_id:
    pitcherid.add(i)

In [21]:
len(pitcherid)

996

Combining FF(Four-Seam Fastball) and FT(Two-Seam Fastball) to FB (Fastball); KN and KC(KnuckleCurve) to KN (Knuckleball)

The reason why we combined FF and FT to FB is that both are the types of a Fastball.
Also, we combined KC with KN because they are similar types and there is not much value counts of KN

In [22]:
data1718_merge.pitch_type = data1718_merge.pitch_type.replace({"FF": "FB", "FT": "FB", "KC": "KN"})

In [23]:
data1718_merge.pitch_type.value_counts()

FB    685370
SL    240439
CH    146135
CU    118919
SI    108393
FC     75883
KN     38839
FS     22126
Name: pitch_type, dtype: int64

modified version of proportion calculator (faster)

Based on the proportion of pitch types thrown by each pitcher, we will weigh each pitching types differently, and they will be reflected as new features for prediction.

Our current weighing scheme for pitch type proportion by year: **2017 : 2018 = 1 : 4**

In [24]:
new_1718 = pd.DataFrame()
new_1718['pitcher_id'] = [i for i in pitcherid]

In [25]:
# for indexing purpose, create a dictionary {pitcher_id: index}
pitcher_locator = dict()
for i, row in new_1718.iterrows():
    pitcher_locator[int(row['pitcher_id'])] = i

In [26]:
# MODIFIED: 'FF' and 'FT' becomes 'FB'; 'KN' and 'KC' are combined into 'KN'
# Get the pitch type proportion for each pitcher, from 2017 data
index = data1718_merge.index
new_1718['FB'] = 0
new_1718['SL'] = 0
new_1718['CH'] = 0
new_1718['CU'] = 0
new_1718['SI'] = 0
new_1718['FC'] = 0
new_1718['FS'] = 0
new_1718['KN'] = 0

In [27]:
'''
for 2017 data, 1 will be added; for 2018 data, 4 will be added
'''
ratio = {2017: 1, 2018: 4}
start = dt.datetime.now()
for _, row in data1718_merge.iterrows():
    new_1718.loc[pitcher_locator[row['pitcher_id']], str(row['pitch_type'])] += ratio[int(row['year'])]

Start: 2022-01-14 22:25:37.001289
End: 2022-01-14 22:29:31.584597

It took... 0:03:54.583844


The following code will calculate the proportion of pitch types thrown by each pitcher

In [28]:
proportion = new_1718.iloc[:, 1:].div(new_1718.iloc[:, 1:].sum(axis=1), axis=0)

In [29]:
proportion['pitcher_id'] = [i for i in pitcherid]
cols = proportion.columns.tolist()
cols = cols[-1:] + cols[:-1]
proportion = proportion[cols]

In [30]:
proportion.head(5)

,pitcher_id,FB,SL,CH,CU,SI,FC,FS,KN
0,466948,0.532710,0.009346,0.355140,0.102804,0.000000,0.000000,0.0,0.0
1,456713,0.660906,0.017281,0.001920,0.197389,0.000000,0.122504,0.0,0.0
2,622608,0.661909,0.197529,0.071480,0.069081,0.000000,0.000000,0.0,0.0
3,501789,0.081851,0.000000,0.000000,0.365362,0.000000,0.552788,0.0,0.0
4,596001,0.316554,0.326886,0.059427,0.077449,0.219685,0.000000,0.0,0.0


In [31]:
'''
IMPORTANT: proportion is the actual proportion of the pitches;
new_1718 contains the raw number of pitches
'''
proportion.to_csv("new_1718.csv")

In [32]:
proportion.iloc[:, 1:].head(5)

,FB,SL,CH,CU,SI,FC,FS,KN
0,0.532710,0.009346,0.355140,0.102804,0.000000,0.000000,0.0,0.0
1,0.660906,0.017281,0.001920,0.197389,0.000000,0.122504,0.0,0.0
2,0.661909,0.197529,0.071480,0.069081,0.000000,0.000000,0.0,0.0
3,0.081851,0.000000,0.000000,0.365362,0.000000,0.552788,0.0,0.0
4,0.316554,0.326886,0.059427,0.077449,0.219685,0.000000,0.0,0.0


In [33]:
new_1718.iloc[:, 1:].head(5)

,FB,SL,CH,CU,SI,FC,FS,KN
0,57,1,38,11,0,0,0,0
1,1721,45,5,514,0,319,0,0
2,5519,1647,596,576,0,0,0,0
3,345,0,0,1540,0,2330,0,0
4,4075,4208,765,997,2828,0,0,0


In [ ]:
# continue from below, if "new_1718.csv" already available
# --------------------------------------------

In [34]:
new_1718 = pd.read_csv("new_1718.csv")

In [35]:
# Using only 2018 year
data18 = data[data['year'] == '2018']

In [36]:
# Merging the data with names
data18 = pd.merge(data18, names, left_on = 'pitcher_id', right_on = 'id')

In [37]:
# Making speed as numeric
data18['speed'] = data18['speed'].str[:-3]
data18['speed'] = data18.speed.astype('int')

In [38]:
# Changing to numeric
number = LabelEncoder()
data18['venue_name'] = number.fit_transform(data18['venue_name'])
data18['condition'] = number.fit_transform(data18['condition'])

We have attempted to add several new features. One is wind direction, that will be labelled by whether the wind condition is for / against / neutral of pitcher. Another one is the pitch code, which indicates how that exact pitch resulted in which in-play: was it out? no out? run? and else.

In [39]:
# function that is used to label:
# 1 if the wind condition is in advantage of hitter (Out);
# 2 if the wind condition is in advantage of pitcher (In);
# 0 if neutral
def wind_direction(row):
    if row['direction'][:1] == 'O':
        return 1
    elif row['direction'][:1] == 'I':
        return 2
    else:
        return 0

In [40]:
# Changind dirction value to 0,1,2
data18['direction'] = data18.apply(wind_direction, axis=1)

In [ ]:
# do the same for pitch codes

In [41]:
data1718['code'].head(5)

0    B
1    F
2    B
3    D
4    C
Name: code, dtype: object

In [42]:
# function that is used to label:
# 1 if in-play; out,
# 2 if in-play; no out,
# 3 if in-play; runs,
# 4 if hit by pitch,
# 0 if else
def last_pitch(row):
    if row['code'] == 'X':
        return 1
    elif row['code'] == 'D':
        return 2
    elif row['code'] == 'E':
        return 3
    elif row['code'] == 'H':
        return 4
    else:
        return 0

In [43]:
data18['code'] = data18.apply(last_pitch, axis=1)

In [44]:
data18['code'].value_counts()

0    596228
1     81801
2     28526
3     15967
4      1922
Name: code, dtype: int64

In [45]:
new_data = pd.DataFrame(data18[['pitch_type', 'b_score', 's_count', 'outs', 'pitch_num', \
                            'on_1b', 'on_2b', 'on_3b', 'batter_id', 'inning', 'p_score', 'p_throws',\
                             'pitcher_id', 'stand', 'top', 'b_count',\
                          'venue_name','speed', 'direction',\
                          'temperature', 'condition', 'code']])

In [46]:
pXs = new_data

pXs is the very final file that includes all the features we will be using to train for pitch type prediction.

In [47]:
pXs = new_data.to_csv("pXs.csv")